# Commercial Banks - Data Extraction Template

This notebook computes the main ratios and performance indicators for commercial banks in Mexico, and outputs a `csv` file for easy manipulation of the data in Excel.


## Basic imports

We'll load the basic libraries and pre-defined functions that will help us easily query TUKAN's API.

In [1]:
import sys
import os
import pandas as pd
import numpy as np
import time
import requests
from datetime import datetime
from dateutil.relativedelta import relativedelta


# We load pre-defined functions
module_path = os.path.abspath(os.path.join('../../'))
if module_path not in sys.path:
    sys.path.append(module_path+"\\utils")
    sys.path.append(module_path+"\\assets")

from tukan_helper_functions import *

## Getting the data

The `get_banks_data` function will help us get the data we need in order to get a complete overview of the Mexican commercial banking system.

In [2]:
def get_banks_data(payload, system_data=False, slice_data=True, change_headers=False):
#system info
  if system_data:
    payload.update({"from":"2015-01-10","categories": {"institutions": ["0c959ae6bc0d063"]}})
    response_data = get_tukan_api_request(payload)
    data = response_data["data"]
    variables = response_data["variables"]
  else:
    if slice_data:
    #most recent data
      response_data = get_tukan_api_request(payload)
      data = response_data["data"]
      variables = response_data["variables"]
      date_recent = data["date"].iloc[0]
    #last year (same month) data
      date_last_year = datetime(date_recent.year, date_recent.month, date_recent.day) - relativedelta(years=1)
      payload.update({"from":date_last_year.strftime("%Y-%m-%d"),"to":date_last_year.strftime("%Y-%m-%d")})
      response_data = get_tukan_api_request(payload)
      data_aux = response_data["data"]
      data = data.append(data_aux)
    #last month data
      date_last_month = datetime(date_recent.year, date_recent.month, date_recent.day) - relativedelta(months=1)
      payload.update({"from":date_last_month.strftime("%Y-%m-%d"),"to":date_last_month.strftime("%Y-%m-%d")})
      response_data = get_tukan_api_request(payload)
      data_aux = response_data["data"]
      data = data.append(data_aux)
    else:
      payload.update({"from":"2015-01-10"})
      response_data = get_tukan_api_request(payload)
      data_aux = response_data["data"]
      data=data_aux

#changing headers
  if change_headers:
    variable_dic = pd.DataFrame(variables)
    variable_dic = variable_dic[["ref", "display_name"]]
    variable_dic = variable_dic.set_index('ref').to_dict()
    data.rename(columns = variable_dic["display_name"], inplace = True)
  return data


The `compute_financial_ratios` function helps us compute the main financial ratios and metrics we will need for our analysis. These include:

*   $\text{ROA}$: return on assets. The sum of LTM profit loss divided by average total assets. 
*   $\text{ROE}$: return on equity. The sum of LTM profit loss divided by average equity.
*   $\text{Coverage ratio}$: loan loss provisions (from balance sheet) divided by the non-performing loan portfolio.
*   $\text{Cost of risk}$: the sum of LTM loan loss expenses divided by the average loan portfolio.
*   $\text{NPL ratio}$: non-performing loan portfolio divided by the total loan portfolio.  
*   $\text{Net interest margin}$: the difference of interest income and interest expenses (LTM) divided by productive assets. Where:

$$\small{\text{Productive assets} = \text{Performing loans} + \text{Cash and cash equivalents} + \text{Financial instruments} + \text{Investments in securities}}$$

In [3]:
def compute_financial_ratios(df):
  '''
  df: the data frame to which we will perform the computations.
  '''

  aux_df = df.copy()
  aux_df.sort_values(by = ['institutions__ref', 'date'], inplace = True)
  aux_df = aux_df.reset_index(drop = True).copy()

  # LTM Profit (loss)
  aux_df.loc[:,'ltm_profit'] = (
      aux_df.groupby(['institutions__ref'])
      .rolling(on="date", window=12, min_periods=12)['41ed2726f53af05']
      .sum()
      .reset_index(drop = True)
    )
  
  # Average assets
  aux_df.loc[:,'ltm_assets'] = (
      aux_df.groupby(['institutions__ref'])
      .rolling(on='date', window=12, min_periods=12)['5fab7445bdba98d']
      .mean()
      .reset_index(drop = True)
    )
  
  # Average equity
  aux_df.loc[:,'ltm_equity'] = (
      aux_df.groupby(['institutions__ref'])
      .rolling(on='date', window=12, min_periods=12)['aaabada5fb35b5e']
      .mean()
      .reset_index(drop = True)
    )
  
  # Total loan portfolio
  aux_df.loc[:,'total_loans'] = (aux_df.loc[:,'b4ea69bfe2b9735'] + aux_df.loc[:,'aa9a4f2f8a95db3'])
  
  # Productive assets
  aux_df.loc[:,'prod_assets'] = (
      aux_df.loc[:,'e45fb5b929acd55'] +
      aux_df.loc[:,'3b64c196a705595'] +
      aux_df.loc[:,'16b0c56b0671f04'] + 
      aux_df.loc[:,'aa9a4f2f8a95db3']
    )
  
  # Captación tradicional
  aux_df.loc[:,'capt_trad'] = (
      aux_df.loc[:,'f26c53c213511d1'] +
      aux_df.loc[:,'893fe1f7da06f33']
    )
  
  # Quitas y castigos
  aux_df.loc[:,'ltm_write_offs'] = (
      aux_df.groupby(['institutions__ref'])
      .rolling(on="date", window=12, min_periods=12)['8a7ec243c69f691']
      .sum()
      .reset_index(drop = True)
    )
  
  # Average productive assets
  aux_df.loc[:,'ltm_prod_assets'] = (
      aux_df.groupby(['institutions__ref'])
      .rolling(on='date', window=12, min_periods=12)['prod_assets']
      .mean()
      .reset_index(drop = True)
    )

  # Average total loans
  aux_df.loc[:,'ltm_total_loans'] = (
      aux_df.groupby(['institutions__ref'])
      .rolling(on='date', window=12, min_periods=12)['total_loans']
      .mean()
      .reset_index(drop = True)
    )

  # LTM (Net interest income)
  aux_df.loc[:,'ltm_nii'] = (
      aux_df.groupby(['institutions__ref'])
      .rolling(on='date', window=12, min_periods=12)['b20c1d56c3cbe96']
      .sum()
      .reset_index(drop = True)
    ) - (
      aux_df.groupby(['institutions__ref'])
      .rolling(on='date', window=12, min_periods=12)['144b301e50c7a67']
      .sum()
      .reset_index(drop = True)
    ) 

  # LTM Loan Loss Expenses
  aux_df.loc[:,'ltm_lle'] = (
      aux_df.groupby(['institutions__ref'])
      .rolling(on="date", window=12, min_periods=12)['4383de47775b541']
      .sum()
      .reset_index(drop = True)
    )
  # LTM Loan Loss Expenses
  aux_df.loc[:,'ltm_lle'] = (
      aux_df.groupby(['institutions__ref'])
      .rolling(on="date", window=12, min_periods=12)['4383de47775b541']
      .sum()
      .reset_index(drop = True)
  )
  # Net commissions
  aux_df.loc[:,'net_com'] = aux_df.loc[:,'9acb0d8ce48d518'] - aux_df.loc[:,'ed1d12a468dbd66']
  #-----------------------------------------------------------------------------
  # Compute the financial ratios
  #-----------------------------------------------------------------------------
  
  aux_df.loc[:,'roa'] = aux_df.loc[:,'ltm_profit'] / aux_df.loc[:,'ltm_assets']
  aux_df.loc[:,'roe'] = aux_df.loc[:,'ltm_profit'] / aux_df.loc[:,'ltm_equity']
  aux_df.loc[:,'cov_ratio'] = -1*(aux_df.loc[:,'d475a2e760d8026'] / aux_df.loc[:,'b4ea69bfe2b9735'])
  aux_df.loc[:,'npl'] = aux_df.loc[:,'b4ea69bfe2b9735'] / aux_df.loc[:,'total_loans']
  aux_df.loc[:,'nim'] = aux_df.loc[:,'ltm_nii'] / aux_df.loc[:,'ltm_prod_assets']
  aux_df.loc[:,'cost_risk'] = aux_df.loc[:,'ltm_lle'] / aux_df.loc[:,'ltm_total_loans']
  aux_df.loc[:,'tda'] =   (aux_df.loc[:,'b4ea69bfe2b9735'] + aux_df.loc[:,'ltm_write_offs']) / (aux_df.loc[:,'total_loans'] + aux_df.loc[:,'ltm_write_offs'])
  aux_df.loc[:,'eficiencia'] = aux_df.loc[:,'e6aa7b4c9299e8f'] / (aux_df.loc[:,'b20c1d56c3cbe96'] - aux_df.loc[:,'144b301e50c7a67'] +  aux_df.loc[:,'net_com']+  aux_df.loc[:,'beb5cfec9761423'] + aux_df.loc[:,'b8f24dd9230d269']) 
  aux_df.loc[:,'icor'] = aux_df.loc[:,'d475a2e760d8026'] / aux_df.loc[:,'b4ea69bfe2b9735']

  #npl_gov - NPL Government Loan Portoflio
  aux_df.loc[:,'npl_gov'] = aux_df.loc[:,'5a454d2ce3208f6'] / (aux_df.loc[:,'1f445d824be3409'] +  aux_df.loc[:,'5a454d2ce3208f6']) 

  #npl_commercial - NPL Corporate Loan Portoflio
  aux_df.loc[:,'npl_commercial'] = aux_df.loc[:,'d30540164451755'] / (aux_df.loc[:,'d30540164451755'] +  aux_df.loc[:,'fe3b7ecbebca755']) 

  #npl_mortages - NPL Mortgage Loan Portoflio
  aux_df.loc[:,'npl_mortages'] = aux_df.loc[:,'a96afb11c29d06f'] / (aux_df.loc[:,'4173bc0829231f9'] +  aux_df.loc[:,'a96afb11c29d06f']) 

  #npl_consum - NPL Consumer Loan Portoflio
  aux_df.loc[:,'npl_consum'] = aux_df.loc[:,'e92557c7fb5563c'] / (aux_df.loc[:,'79e1d987d9e13b1'] +  aux_df.loc[:,'e92557c7fb5563c']) 

  #npl_finent - NPL Financial Entities Loan Portoflio
  aux_df.loc[:,'npl_finent'] = aux_df.loc[:,'8a8891292a2219c'] / (aux_df.loc[:,'a4526a18f4faf4a'] +  aux_df.loc[:,'8a8891292a2219c']) 

  #We drop all of the columns that are in the original DF but not on the new DF
  columns_set = set(aux_df.columns)
  columns_keep = columns_set.difference(set(df.columns))
  
  aux_df = aux_df[['date', 'institutions', 'institutions__ref'] + 
                  list(columns_keep)].copy()

  return aux_df

## Bank groups

Commercial banks in Mexico are usually looked at through the following groups. In this section we define a `dictionary` that will help us aggregate statistics based on this classification.

In [4]:
bank_groups = {'ABC Capital': 'CONSUMO',
  'Accendo Banco': 'COMERCIALES',
  'American Express Bank México': 'OTROS',
  'BBVA Bancomer': 'G7',
  'BNP Paribas México': 'OTROS',
  'BanCoppel': 'TIENDAS',
  'Banamex': 'G7',
  'Banca Afirme': 'COMERCIALES',
  'Banca Mifel': 'COMERCIALES',
  'Banco Actinver': 'PRIVADA',
  'Banco Autofin México': 'CONSUMO',
  'Banco Azteca': 'TIENDAS',
  'Banco Bancrea': 'COMERCIALES',
  'Banco Base': 'PRIVADA',
  'Banco Compartamos': 'CONSUMO',
  'Banco Credit Suisse México': 'OTROS',
  'Banco Finterra': 'COMERCIALES',
  'Banco Forjadores': 'CONSUMO',
  'Banco Inbursa': 'G7',
  'Banco Inmobiliario Mexicano': 'CONSUMO',
  'Banco Invex': 'PRIVADA',
  'Banco Monex': 'PRIVADA',
  'Banco Multiva': 'COMERCIALES',
  'Banco PagaTodo': 'CONSUMO',
  'Banco S3 Caceis México': 'OTROS',
  'Banco de Inversión Afirme': 'OTROS',
  'Banco del Bajío': 'COMERCIALES',
  'Bank of America México': 'OTROS',
  'Bank of China México': 'OTROS',
  'Bankaool': 'OTROS',
  'Banorte': 'G7',
  'Bansi': 'COMERCIALES',
  'Barclays Bank México': 'OTROS',
  'Cibanco': 'PRIVADA',
  'Consubanco': 'TIENDAS',
  'Deutsche Bank México': 'OTROS',
  'Fundación Dondé Banco': 'CONSUMO',
  'HSBC México': 'G7',
  'Industrial and Commercial Bank of China México': 'OTROS',
  'Intercam Banco': 'PRIVADA',
  'J.P. Morgan México': 'OTROS',
  'KEB Hana México': 'OTROS',
  'MUFG Bank México': 'OTROS',
  'Mizuho Bank México': 'OTROS',
  'Regional': 'COMERCIALES',
  'Sabadell': 'COMERCIALES',
  'Santander México': 'G7',
  'Scotiabank Inverlat': 'G7',
  'Shinhan de México': 'OTROS',
  'Ve Por Más': 'COMERCIALES',
  'Volkswagen Bank': 'CONSUMO',
  'Banco Ahorro Famsa': 'OTROS', 
  'Banco Interacciones': 'OTROS',
  'Banco Wal-Mart de México Adelante':'OTROS'}

## Balance Sheet data

In this section we structure the `payload` to query balance sheet items related to our analysis.

In [5]:
payload = {
    "type": "data_table",
    "operation": "sum",
    "language": "es",
    "categories": {
        "institutions": "all"
    },
    "request": [
        {
            "table": "mex_cnbv_cb_balance_sheet",
            "variables": [
                "5fab7445bdba98d",
                "3b64c196a705595",
                "16b0c56b0671f04",
                "e45fb5b929acd55",
                "aa9a4f2f8a95db3",
                "a4526a18f4faf4a",
                "1f445d824be3409",
                "79e1d987d9e13b1",
                "4173bc0829231f9",
                "b4ea69bfe2b9735",
                "8a8891292a2219c",
                "5a454d2ce3208f6",
                "e92557c7fb5563c",
                "a96afb11c29d06f",
                "3f1b1935ca21bbd",
                "f26c53c213511d1",
                "893fe1f7da06f33",
                "d475a2e760d8026",               
                "aaabada5fb35b5e",
                "fe3b7ecbebca755",
                "d30540164451755"
            ]
        }
    ]
}

payload_banks_hist = payload.copy()

payload_system = payload.copy()

And now we query historical data for the whole system, and only recent information for all commercial banks.

In [6]:
genbal_banks_hist = get_banks_data(payload_banks_hist, system_data=False, slice_data=False, change_headers=False)

Success getting the data


In [7]:
# genbal_system = get_banks_data(payload_banks_hist, system_data=True, slice_data=False, change_headers=False)

## Income Statement data

In this section we structure the `payload` to query income statement items related to our analysis.

In [8]:
payload = {
    "type": "data_table",
    "operation": "sum",
    "language": "es",
    "categories": {
        "institutions": "all"
    },
    "request": [
        {
            "table": "mex_cnbv_cb_income_statement",
            "variables": [
                "b20c1d56c3cbe96",
                "144b301e50c7a67",
                "4383de47775b541",
                "c527586c5f852ce",
                "9acb0d8ce48d518",
                "ed1d12a468dbd66",
                "b8f24dd9230d269",
                "e6aa7b4c9299e8f",
                "41ed2726f53af05",
                "beb5cfec9761423",
                "8a7ec243c69f691"
            ]
        }
    ]
}
payload_banks_hist = payload.copy()

payload_system = payload.copy()

And now we query historical data for the whole system, and only recent information for all commercial banks.

In [9]:
incsta_banks_hist = get_banks_data(payload_banks_hist, system_data=False, slice_data=False, change_headers=False)

Success getting the data


In [41]:
# incsta_system = get_banks_data(payload_banks_hist, system_data=True, slice_data=False, change_headers=False)

Success getting the data


## CET1 Ratio

In this section we are interested in querying capital adequacy ratios. Mainly, the CET1 ratio.

In [10]:
payload = {
    "type": "data_table",
    "operation": "sum",
    "language": "es",
    "group_by": [
        "institutions"
    ],
    "categories": {
        "institutions": "all"
    },
    "request": [
        {
            "table": "mex_cnbv_cb_capital_ratios",
            "variables": [
                "2440b46b217f949"
            ]
        }
    ]
}
payload_banks = payload.copy()
payload_system = payload.copy()

In [11]:
icap_banks =  get_banks_data(payload_banks, system_data=False, change_headers=False)

Success getting the data
Success getting the data
Success getting the data


In [12]:
icap_system = get_banks_data(payload_system, system_data=True, change_headers=False)

Success getting the data


## Operational data

In this section we query data related to operational statistics, such as: employees, ATMs, etc.

In [54]:
payload = {
    "type": "data_table",
    "operation": "sum",
    "language": "es",
    "group_by": [
        "geography",
        "institutions"
    ],
    "categories": {
        "geography": [
            "b815762a2c6a283"
        ],
        "institutions": "all"
    },
    "request": [
        {
            "table": "mex_cnbv_cb_operational",
            "variables": [
                "ad1841c2825df92",
                "c3f78436c051fcd",
                "2eee037c67fbe81",
                "6ce544361c47af9",
                "21879990c9338ac"
            ]
        }
    ]
}

In [55]:
ops_banks =  get_banks_data(payload, system_data=False, change_headers=False)
ops_banks.drop(columns=['geography__ref', 'geography'],inplace=True)
ops_banks.fillna(0, inplace = True)
ops_banks.loc[:,'employees'] = ops_banks.loc[:,'c3f78436c051fcd'] + (ops_banks.loc[:,'21879990c9338ac'])

Success getting the data
Success getting the data
Success getting the data


Now we compute the aggregate for the whole system.

In [56]:
# First we get the data by groups

ops_banks_groups = ops_banks.copy()
ops_banks_groups.replace({"institutions":bank_groups}, inplace = True)
ops_banks_groups.loc[:,"institutions__ref"] = ops_banks_groups["institutions"]
ops_banks_groups = ops_banks_groups.groupby(["institutions", "institutions__ref", "date"]).sum().reset_index()

# Then we get the data for the whole system
ops_banks_system = ops_banks.groupby(["date"]).sum().reset_index()
ops_banks_system.loc[:,"institutions"] = "Instituciones de banca múltiple"
ops_banks_system.loc[:,"institutions__ref"] = "0c959ae6bc0d063"

# And we append with the original dataset.
ops_banks_cons = ops_banks.append(ops_banks_system)
ops_banks_cons = ops_banks_cons.append(ops_banks_groups)
ops_banks_cons.drop_duplicates(inplace = True)


## Compute ratios and generate file

First, we merge the data and group statistics by the banking groups defined above. Then we compute the data for the whole system.

In [74]:
hist_data = genbal_banks_hist.merge(incsta_banks_hist, how='left',on=['date', 'institutions__ref', 'institutions'])
bank_data = hist_data.copy()
bank_data = bank_data[bank_data["institutions__ref"] != "0c959ae6bc0d063"]
bank_data.replace({"institutions": bank_groups}, inplace = True)
bank_data.loc[:,"institutions__ref"] = bank_data["institutions"]
bank_data = bank_data.groupby(["institutions", "institutions__ref", "date"]).sum().reset_index()

system = hist_data[hist_data["institutions__ref"] == "0c959ae6bc0d063"].copy()
system.drop_duplicates(inplace = True)
system_financial_ratios = compute_financial_ratios(system)

Now, we compute the financial ratios for individual banks and bank groups.

In [75]:
banks_hist = genbal_banks_hist.merge(incsta_banks_hist, how='left',on=['date', 'institutions__ref', 'institutions'])

banks_hist = banks_hist.append(bank_data)
banks_hist.drop_duplicates(inplace = True)
financial_ratios = compute_financial_ratios(banks_hist)

Finally, we merge all the calculations we've made into a single dataframe.

In [76]:
#BANKS
# merging dfs
banks_hist_data = pd.merge(financial_ratios, banks_hist, how = "left", on = ['date', 'institutions__ref', 'institutions'])
banks_hist_data = pd.merge(banks_hist_data, icap_banks, how = "left", on = ['date', 'institutions__ref', 'institutions'])
banks_hist_data = pd.merge(banks_hist_data, ops_banks_cons, how = "left", on = ['date', 'institutions__ref', 'institutions'])

# # filtering to keep most recent data
max_date = banks_hist_data["date"].max()
lag_1y_date = max_date - relativedelta(years = 1)
lag_1m_date = max_date - relativedelta(months = 1)
lag_2m_date = max_date - relativedelta(months = 2)
lag_1y_date_1m = lag_1m_date - relativedelta(years = 1)
lag_1y_date_2m = lag_2m_date - relativedelta(years = 1)

# # final DF
banks_final_data = banks_hist_data[banks_hist_data["date"].isin([max_date, lag_1m_date, lag_1y_date, lag_2m_date, lag_1y_date_1m, lag_1y_date_2m])].reset_index(drop=True)

In [77]:
#SYSTEM
# merging dfs
system_data = pd.merge(system_financial_ratios, system, how = "left", on = ['date', 'institutions__ref', 'institutions'])
system_data = pd.merge(system_data, icap_system, how = "left", on = ['date', 'institutions__ref', 'institutions'])

Finally, we colect the "human readable" variable names so we can export the dataframe with intuitive column names.

In [80]:
table_dict = get_table_dictionary('mex_cnbv_cb_balance_sheet')
table_dict = table_dict.append(get_table_dictionary('mex_cnbv_cb_income_statement'))
table_dict = table_dict.append(get_table_dictionary('mex_cnbv_cb_capital_ratios'))
table_dict = table_dict.append(get_table_dictionary('mex_cnbv_cb_operational'))
variable_dic = table_dict[["ref", "display_name"]]
variable_dic = variable_dic.set_index('ref').to_dict()
banks_final_data.rename(columns = variable_dic["display_name"], inplace = True)
system_data.rename(columns = variable_dic["display_name"], inplace = True)

And finally we ensure that the columns are stored in the precise order we want them.

In [81]:
banks_final_data = banks_final_data[['date', 'institutions', 'institutions__ref', 'total_loans',
       'npl_consum', 'roe', 'ltm_lle', 'npl_gov', 'eficiencia', 'tda', 'icor',
       'nim', 'ltm_total_loans', 'ltm_assets', 'capt_trad', 'ltm_profit',
       'ltm_prod_assets', 'roa', 'cost_risk', 'npl', 'net_com', 'ltm_equity',
       'prod_assets', 'ltm_write_offs', 'npl_finent', 'npl_mortages',
       'ltm_nii', 'cov_ratio', 'npl_commercial', 'Inversiones en valores',
       'Cartera de crédito vigente a entidades gubernamentales',
       'Efectivo y equivalentes de efectivo', 'Pasivos',
       'Cartera de crédito hipotecaria vigente',
       'Cartera de crédito vencida a entidades gubernamentales', 'Activos',
       'Cartera de crédito de consumo vigente', 'Depósitos a plazo',
       'Cartera de crédito vencida a entidades financieras',
       'Cartera de crédito vigente a entidades financieras',
       'Cartera de crédito hipotecaria vencida', 'Cartera de crédito vigente',
       'Capital contable', 'Cartera de crédito vencida',
       'Cartera de crédito empresarial vencida',
       'Estimación preventiva para riesgos crediticios',
       'Operaciones con valores y derivados (activo)',
       'Cartera de crédito de consumo vencida',
       'Depósitos de exigibilidad inmediata',
       'Cartera de crédito empresarial vigente', 'Gastos por intereses',
       'Utilidad (pérdida) neta',
       'Reservas adicionales para riesgos crediticios', 'Quitas y castigos',
       'Comisiones y tarifas cobradas', 'Ingresos por intereses',
       'Resultado por intermediacion', 'Otros ingresos financieros',
       'Margen financiero', 'Gastos de administración',
       'Comisiones y tarifas pagadas', 'Índice de capitalización total (ICAP)',
       'Empleados dependientes de la razón social', 'Sucursales',
       'Transacciones en cajeros automáticos', 'Cajeros automáticos',
       'Empleados suministrados por otra razón social', 'employees']]

In [82]:
system_data = system_data[['date', 'institutions', 'institutions__ref', 'total_loans',
       'npl_consum', 'roe', 'ltm_lle', 'npl_gov', 'eficiencia', 'tda', 'icor',
       'nim', 'ltm_total_loans', 'ltm_assets', 'capt_trad', 'ltm_profit',
       'ltm_prod_assets', 'roa', 'cost_risk', 'npl', 'net_com', 'ltm_equity',
       'prod_assets', 'ltm_write_offs', 'npl_finent', 'npl_mortages',
       'ltm_nii', 'cov_ratio', 'npl_commercial', 'Inversiones en valores',
       'Cartera de crédito vigente a entidades gubernamentales',
       'Efectivo y equivalentes de efectivo', 'Pasivos',
       'Cartera de crédito hipotecaria vigente',
       'Cartera de crédito vencida a entidades gubernamentales', 'Activos',
       'Cartera de crédito de consumo vigente', 'Depósitos a plazo',
       'Cartera de crédito vencida a entidades financieras',
       'Cartera de crédito vigente a entidades financieras',
       'Cartera de crédito hipotecaria vencida', 'Cartera de crédito vigente',
       'Capital contable', 'Cartera de crédito vencida',
       'Cartera de crédito empresarial vencida',
       'Estimación preventiva para riesgos crediticios',
       'Operaciones con valores y derivados (activo)',
       'Cartera de crédito de consumo vencida',
       'Depósitos de exigibilidad inmediata',
       'Cartera de crédito empresarial vigente', 'Gastos por intereses',
       'Utilidad (pérdida) neta',
       'Reservas adicionales para riesgos crediticios', 'Quitas y castigos',
       'Comisiones y tarifas cobradas', 'Ingresos por intereses',
       'Resultado por intermediacion', 'Otros ingresos financieros',
       'Margen financiero', 'Gastos de administración',
       'Comisiones y tarifas pagadas',
       'Índice de capitalización total (ICAP)']]

And export the final data to a `csv` file.

In [84]:
#csv
banks_final_data.to_csv('data_outputs/banks_final_data.csv', index=False, encoding = "iso-8859-1")
system_data.to_csv('data_outputs/system_final_data.csv', index=False, encoding = "iso-8859-1")